Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [14]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
from stockanalyzer import analyze_stock, stock_data_setup
import importlib

In [15]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

# Initialize an empty list to store the stocks
stocks_list = []

df_portfolio = pd.read_csv('config/portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

### Stocks Data

In [ ]:
analysis = analyze_stock(
	ticker= ['MSFT'], 
	dic_files=dic_files,
	dic_assistants=dic_assistants
)

In [ ]:
print(analysis)

In [16]:
ticker = ['MSFT']

client = OpenAI()

file_stocks = stock_data_setup(client=client, ticker=ticker, type='price', dic_files=dic_files)
file_cashflow = stock_data_setup(client=client, ticker=ticker, type='cash', dic_files=dic_files)
file_income_stmt = stock_data_setup(client=client, ticker=ticker, type='income', dic_files=dic_files)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
	'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
	json.dump(dic_assistants, json_file)
	print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
	client = client, 
	new=False,
	assistant_name = 'fin_analyst',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

fin_consultant = AgentHandler(
	client = client,
	new=False,
	assistant_name = 'fin_consultant',
	dic_file = dic_assistants,
	dic_file_name = 'assistants.json',
	dic_file_path='config/'
	)

# prompt_start = f"This is your client. I want you to advice me on the stocks I list below.\
# 	I want to know the stocks' performance, if they're overvalued or undervalued, and whether or not they'll be a good investment.\
# 	I am looking for a long term (5+ years) investment and I have a relatively high risk tolerance.\
# 	Stocks: {ticker}"

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_stocks.csv, file id: file-6P9s8mTSkBIKvCTXLEryEvrT has been deleted
file name: df_stocks.csv is uploaded, new file id: file-8z1lk9qFZtlBJLtUS5vtz9LV
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_cashflow.csv, file id: file-ddpmP5flTAwLLBEJiBHWMvct has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-p9NHEX0QLRgWiHQW5LEKLV10
openai_upload_files/openai_files.json file has been updated


d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)


file name: df_income_stmt.csv, file id: file-lCMKoHSqCMQpns7NX01RQFh7 has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-8JyWbwBzJI1pPd6YHcWh8eIk
openai_upload_files/openai_files.json file has been updated
assistants.json file has been updated
Assistant has been updated, name: fin_analyst, id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
Assistant has been updated, name: fin_consultant, id: asst_mPlBQ4ZF8YIF238DWqWRhV4U


In [17]:
prompt_start = f"This is the financial consultant.\
	The client wants advice on the following stocks.\
	I need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\
	You will be provided with the data of these stocks, use them as you see fit.\
	Stocks: {ticker}"

thread = ThreadManager(
	client=client,
	prompt=prompt_start
)

thread.run_thread(
	assistant=fin_analyst,
	prompt=prompt_start,
	attachments=[file_stocks.file_id, file_cashflow.file_id, file_income_stmt.file_id]
)

next_prompt = thread.last_message

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > Here are the information gathered from the financial analyst:

These are the stocks the client wants advice on:
- MSFT (Microsoft Corporation)

Let's review the data provided and analyze the stock performance for MSFT.

We'll begin with checking the contents of the files uploaded to extract relevant data for MSFT.
assistant > code_interpreter

import pandas as pd

# Load the files to check their contents
file_1 = '/mnt/data/file-8JyWbwBzJI1pPd6YHcWh8eIk'
file_2 = '/mnt/data/file-p9NHEX0QLRgWiHQW5LEKLV10'
file_3 = '/mnt/data/file-8z1lk9qFZtlBJLtUS5vtz9LV'

# Reading files and checking the first few rows to understand the data
df1 = pd.read_csv(file_1)
df2 = pd.read_csv(file_2)
df3 = pd.read_csv(file_3)

df1_head = df1.head()
df2_head = df2.head()
df3_head = df3.head()

df1_shape = df1.shape
df2_shape = df2.shape
df3_shape = df3.shape

df1_head, df2_head, df3_head, df1_shape, df2_shape, df3_shape
assistant > Here are the information gathered from the financial analyst:

Thes

In [18]:
next_prompt = 'This message is for testing. Can you give a short sample response.'

thread.run_thread(
	assistant=fin_consultant,
	prompt=next_prompt
)


assistant > ### Stock Recommendation: Microsoft Corporation (MSFT)

**Overview:**
- **Most Recent Close Price**: \$405.72 (as of 2024-09-09)
- **Recent Financial Highlights (2024-06-30)**:
  - **Revenue**: \$245.12 billion
  - **Net Income**: \$88.14 billion
  - **Normalized EBITDA**: \$133.56 billion

**Pros:**
- Strong financial performance and consistent revenue growth.
- Market leader in software and cloud computing.
- Increased operational efficiency.

**Cons:**
- Potential regulatory risks.
- Market saturation in some segments.

**Recommendation:**
Microsoft (MSFT) is a strong BUY due to its solid financial health and steady growth, making it a good fit for a long-term growth-focused investment strategy.get_last_message initiated
message_id: msg_WdEC5hR92LiKUi0OIps2DihQ, assistant_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
asst_id = blank
new_message: ### Stock Recommendation: Microsoft Corporation (MSFT)

**Overview:**
- **Most Recent Close Price**: \$405.72 (as of 2024-09-09)
- **Recen

In [22]:
thread.messages
client.beta.threads.messages.list(thread_id=thread.thread_id).data
# messages_data[0]
# type(file_cashflow)

[Message(id='msg_WdEC5hR92LiKUi0OIps2DihQ', assistant_id='asst_mPlBQ4ZF8YIF238DWqWRhV4U', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='### Stock Recommendation: Microsoft Corporation (MSFT)\n\n**Overview:**\n- **Most Recent Close Price**: \\$405.72 (as of 2024-09-09)\n- **Recent Financial Highlights (2024-06-30)**:\n  - **Revenue**: \\$245.12 billion\n  - **Net Income**: \\$88.14 billion\n  - **Normalized EBITDA**: \\$133.56 billion\n\n**Pros:**\n- Strong financial performance and consistent revenue growth.\n- Market leader in software and cloud computing.\n- Increased operational efficiency.\n\n**Cons:**\n- Potential regulatory risks.\n- Market saturation in some segments.\n\n**Recommendation:**\nMicrosoft (MSFT) is a strong BUY due to its solid financial health and steady growth, making it a good fit for a long-term growth-focused investment strategy.'), type='text')], created_at=1725952118, incomplete_at=None, incomplete_details=None,

In [21]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]

a = thread.messages[0]

thread.messages = []

thread.messages.append(a)

thread.get_last_message()
thread.messages

get_last_message initiated
message_id: msg_WdEC5hR92LiKUi0OIps2DihQ, assistant_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
asst_id = blank
new_message: ### Stock Recommendation: Microsoft Corporation (MSFT)

**Overview:**
- **Most Recent Close Price**: \$405.72 (as of 2024-09-09)
- **Recent Financial Highlights (2024-06-30)**:
  - **Revenue**: \$245.12 billion
  - **Net Income**: \$88.14 billion
  - **Normalized EBITDA**: \$133.56 billion

**Pros:**
- Strong financial performance and consistent revenue growth.
- Market leader in software and cloud computing.
- Increased operational efficiency.

**Cons:**
- Potential regulatory risks.
- Market saturation in some segments.

**Recommendation:**
Microsoft (MSFT) is a strong BUY due to its solid financial health and steady growth, making it a good fit for a long-term growth-focused investment strategy.
message_id: msg_UdcAcnG9rb2AlHzCTHlnw9w6, assistant_id: None
message.assistant_id != asst_id, record previous message first
['### Stock Recommendation

[{'message_id': 'msg_KYlMhdeNqW7wDvLeEPrqTczS',
  'assistant_id': None,
  'created_at': 1725952004,
  'file_ids': [],
  'role': 'user',
  'run_id': None,
  'message_text': "This is the financial consultant.\tThe client wants advice on the following stocks.\tI need you to provide me your analysis on these stocks so I can provide the appropriate recommendations.\tYou will be provided with the data of these stocks, use them as you see fit.\tStocks: ['MSFT']"},
 {'message_id': 'msg_WdEC5hR92LiKUi0OIps2DihQ',
  'assistant_id': 'asst_mPlBQ4ZF8YIF238DWqWRhV4U',
  'created_at': 1725952118,
  'file_ids': [],
  'role': 'assistant',
  'run_id': 'run_8ogrK70oxu3e1faIB9smZwdj',
  'message_text': '### Stock Recommendation: Microsoft Corporation (MSFT)\n\n**Overview:**\n- **Most Recent Close Price**: \\$405.72 (as of 2024-09-09)\n- **Recent Financial Highlights (2024-06-30)**:\n  - **Revenue**: \\$245.12 billion\n  - **Net Income**: \\$88.14 billion\n  - **Normalized EBITDA**: \\$133.56 billion\n\n**

In [38]:
# existing_message_id = [dic['message_id'] for dic in thread.messages]
asst_id = ''

for message in messages_data:

	print(f'message_id: {message.id}, asst_id: {message.assistant_id}')

	if message.id in existing_message_id:
		print('id exists')

	else:
		# print('id doesnt exist')
		if asst_id == '':
			print('asst_id = blank')

			asst_id = message.assistant_id

		elif message.assistant_id == asst_id:

			print('message.assistant_id == asst_id, proceed normally')

		else:

			print('message.assistant_id != asst_id, start a new round of message')

message_id: msg_0YI1I3hF8OTzcEOG98L7Yi1V, asst_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
asst_id = blank
message_id: msg_QAPBtXXNgWnzX53d7SuW9dfQ, asst_id: None
message.assistant_id != asst_id, start a new round of message
message_id: msg_QVoKLLa1XshONulTZ3ZGpgwI, asst_id: asst_mPlBQ4ZF8YIF238DWqWRhV4U
message.assistant_id == asst_id, proceed normally
message_id: msg_H8VId2krDKV9oKD0G0ao59lF, asst_id: None
message.assistant_id != asst_id, start a new round of message
message_id: msg_Q5q5yM7L3n4Ng7F77GADFKON, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_WJz18bE75m6GBxFfevGa0tgp, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_itcdIzYvUErWcQaIbUeoNEAx, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != asst_id, start a new round of message
message_id: msg_uZbO2pgmoVS0p4i28XPKymwo, asst_id: asst_Mqf9cO1sDTOd4UMYwcFfIQrA
message.assistant_id != as

In [23]:
thread.delete_thread()

thread: thread_kKQe7ycVaA9jEmvZM0YNqZ6f has been deleted.
